<a href="https://colab.research.google.com/github/thisisRMak/2025-tech16-LLM/blob/main/Tech16_Lecture_3_class_code_and_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install llama-index
%pip install pypdf

In [3]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')
client = OpenAI(api_key=open_ai_key)

In [4]:
!wget https://privatewealth.goldmansachs.com/outlook/2025-isg-outlook.pdf

--2025-02-20 01:36:43--  https://privatewealth.goldmansachs.com/outlook/2025-isg-outlook.pdf
Resolving privatewealth.goldmansachs.com (privatewealth.goldmansachs.com)... 23.192.230.149, 23.192.230.134
Connecting to privatewealth.goldmansachs.com (privatewealth.goldmansachs.com)|23.192.230.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘2025-isg-outlook.pdf.3’

2025-isg-outlook.pd     [  <=>               ]   6.88M  22.6MB/s    in 0.3s    

2025-02-20 01:36:44 (22.6 MB/s) - ‘2025-isg-outlook.pdf.3’ saved [7209986]



In [5]:
import os
os.environ["OPENAI_API_KEY"] = open_ai_key

In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [7]:
documents = SimpleDirectoryReader('./').load_data('2025-isg-outlook.pdf')
documents

Loading files: 100%|██████████| 1/1 [00:08<00:00,  8.08s/file]


[Document(id_='eca796fa-787b-42c3-ae7f-0f01eb7f63a5', embedding=None, metadata={'page_label': 'i', 'file_name': '2025-isg-outlook.pdf', 'file_path': '/content/2025-isg-outlook.pdf', 'file_type': 'application/pdf', 'file_size': 7209986, 'creation_date': '2025-02-19', 'last_modified_date': '2025-02-19'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Outlook\nKeep On Truckin’\nInvestment Strategy Group | January 2025\nWealth Management\nCan’t nothin’ hold me back, nothin’\nI’ll keep right on, right on truckin’\n – Lyrics from “Keep On Truckin’” by Eddie Kendricks \n#1 on Billboard Hot 100 and R&B Singles Chart (1973)', path=None, url=None, 

In [8]:
documents[10] # see page 10

Document(id_='956e8a91-4464-4f1d-9e66-1f5e0814776b', embedding=None, metadata={'page_label': '9', 'file_name': '2025-isg-outlook.pdf', 'file_path': '/content/2025-isg-outlook.pdf', 'file_type': 'application/pdf', 'file_size': 7209986, 'creation_date': '2025-02-19', 'last_modified_date': '2025-02-19'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='9Outlook Investment Strategy Group\n SECTION II: DIVERGING PATHS\n58  2025 Global  \nEconomic Outlook\n  Despite divergent paths, most economies will \nascend toward higher elevations this year.  \n60  United States\n64  Eurozone\n66  United Kingdom\n67 Japan\n68  Emerging Markets\n SECTION III:

In [9]:
index = VectorStoreIndex.from_documents(documents)

In [10]:
query_engine = index.as_query_engine() # this will convert query and document to numerical representation and pass it to model

In [11]:
response = query_engine.query("What is the outlook for 2025 for US GDP?")

In [12]:
print(response)

The outlook for 2025 for US GDP is expected to show solid growth, with real GDP expanding by 2.3%.


In [13]:
response = query_engine.query("What is the recipe of apple pie?")

In [14]:
print(response)

I'm unable to provide the recipe for apple pie as it is not mentioned in the provided context information.


In [15]:
response

Response(response="I'm unable to provide the recipe for apple pie as it is not mentioned in the provided context information.", source_nodes=[NodeWithScore(node=TextNode(id_='e394c0b4-2741-42ec-8aaa-c739797ef8a1', embedding=None, metadata={'page_label': '6', 'file_name': '2025-isg-outlook.pdf', 'file_path': '/content/2025-isg-outlook.pdf', 'file_type': 'application/pdf', 'file_size': 7209986, 'creation_date': '2025-02-19', 'last_modified_date': '2025-02-19'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f556fa0c-b5e7-439c-9d70-226af985f683', node_type='4', metadata={'page_label': '6', 'file_name': '2025-isg-outlook.pdf', 'file_path': '/content/2025-isg-outlook.pdf', 'file_type': 'application/pdf', 'file_

# Adding Memory

In [16]:
response = query_engine.query("What is the outlook for developping markets?")
print(response)

Developing markets are expected to have mid-single-digit returns, with the volatility in this market being the greatest among all markets. There is a 55% probability assigned to the base case scenario, a 25% probability to the upside scenario, and a 20% probability to the downside scenario for emerging market equities. On a probability-weighted basis, emerging market equities have the second-lowest expected return after the UK.


In [17]:
# Models don't have memory, so if we want a chatbot or chat engine that can respond to multiple related questions, we need to give the system memory

response = query_engine.query("What did I just ask you? Give me the exact question")
print(response)

What did I just ask you? Give me the exact question


In [18]:
# In order to give the model memory, we need to create a different engine

from re import VERBOSE
chat_engine = index.as_chat_engine(chat_mode="openai", verbose=True) # other chat_mode's include Simple, React, condense, among others
response = chat_engine.chat("Hello there")
print(response)

Added user message to memory: Hello there
Hi! How can I assist you today?


In [19]:
response = chat_engine.chat("What is the likelihood of a recession in 2025?")
print(response)

Added user message to memory: What is the likelihood of a recession in 2025?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"What is the likelihood of a recession in 2025?"}
Got output: The likelihood of a recession in 2025 is 20%, which is close to the long-term average but below the current consensus.

The likelihood of a recession in 2025 is estimated to be around 20%, which is close to the long-term average but below the current consensus.


In [20]:
response = chat_engine.chat("Can you expand on that?")
print(response)

Added user message to memory: Can you expand on that?
The likelihood of a recession in 2025 being estimated at 20% means that there is a moderate chance of an economic downturn occurring that year. This percentage is close to the long-term average probability of a recession but is slightly lower than the current consensus among experts and analysts. Factors such as economic indicators, market trends, and global events can influence the probability of a recession in 2025. It's important to monitor these factors closely to assess the potential risks and prepare accordingly.


In [21]:
response = chat_engine.chat("What did I just ask you?")
print(response)

Added user message to memory: What did I just ask you?
You asked about the likelihood of a recession in 2025.


# Chunking

In [22]:
from llama_index.core import Settings


Settings.chunk_size = 1000
Settings.chunk_overlap = 50

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
index = VectorStoreIndex.from_documents(documents)

# Convert the VectorStoreIndex object into a query engine. This query engine can be used to
# perform semantic searches on the index, matching natural language queries to the most relevant
# documents in the index.
query_engine = index.as_query_engine()

# Use the query engine to search for documents that are relevant to the query
# from the indexed documents based on the semantic understanding of the query.
response = query_engine.query("What is the 2025 outlook for US GDP?")

# Print the response obtained from the query. This will display the result of the semantic search,
# showing the information or documents that best match the query about the 2025 outlook.
print(response)


The 2025 outlook for US GDP is expected to expand by 2.3% according to the provided context information.


# Vector DB

In [23]:
%pip install llama-index-vector-stores-weaviate

In [24]:
!pip install llama-index-vector-stores-chroma

In [25]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

chroma_client = chromadb.PersistentClient()
chroma_collection = chroma_client.create_collection("tech16example")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [26]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./").load_data("2025-isg-outlook.pdf")
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)
query_engine = index.as_query_engine()
response = query_engine.query("What is the 2025 outlook for US GDP?")
print(response)

Loading files: 100%|██████████| 1/1 [00:14<00:00, 14.38s/file]


The 2025 outlook for US GDP is an expansion of 2.3% according to the provided context information.


# YouTube

## TODO: Debug the youtube import error


In [27]:
!pip install youtube-transcript-api

In [28]:
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader

loader = YoutubeTranscriptReader()
documents = loader.load_data(
    ytlinks=["https://www.youtube.com/watch?v=i3OYlaoj-BM"]
)

ModuleNotFoundError: No module named 'llama_index.readers.youtube_transcript'

In [29]:
index = VectorStoreIndex.from_documents(documents)

# Convert the VectorStoreIndex object into a query engine. This query engine can be used to
# perform semantic searches on the index, matching natural language queries to the most relevant
# documents in the index.
query_engine = index.as_query_engine()

# Use the query engine to search for documents that are relevant to the query
# from the indexed documents based on the semantic understanding of the query.
response = query_engine.query("What is the 2025 outlook for US GDP?")

# Print the response obtained from the query. This will display the result of the semantic search,
# showing the information or documents that best match the query about the 2025 outlook.
print(response)

The 2025 outlook for US GDP is an expansion of 2.3% according to the provided context information.


# *HW3*

Uploaded Captions from Lecture 1,2,3 to a folder labelled "Tech16_Captions"

In [30]:
documents = SimpleDirectoryReader('./Tech16_Captions').load_data('Lecture?.txt')
documents


Loading files: 100%|██████████| 3/3 [00:00<00:00, 1665.95file/s]


[Document(id_='89e6ff33-5a33-4f4f-85f9-cf887495f16f', embedding=None, metadata={'file_path': '/content/Tech16_Captions/Lecture1.txt', 'file_name': 'Lecture1.txt', 'file_type': 'text/plain', 'file_size': 135926, 'creation_date': '2025-02-20', 'last_modified_date': '2025-02-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text="Lecture 1 Captions\r\n\r\nCathal Flanagan: Okay, share my screen.\r\n0:00\r\nCathal Flanagan: Excellent. Okay. Hopefully, folks can can see my screen. Welcome. Welcome to the 1st\r\n0:17\r\nCathal Flanagan: of many classes of tech 16 for this winter.\r\n0:28\r\nCathal Flanagan: llms for business with python. Really exc

In [31]:
index = VectorStoreIndex.from_documents(documents)
chat_engine = index.as_chat_engine(chat_mode="openai", verbose=True)

In [32]:
response = chat_engine.chat("Why do we not use LLMs in enterprise?")
print(response)

Added user message to memory: Why do we not use LLMs in enterprise?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"Why do we not use LLMs in enterprise?"}
Got output: LLMs are not extensively used in enterprise because in scenarios where a single retrieval system or model provider is already established and being utilized, the addition of a LLM like Lama Index may not provide significant value. It is often simpler and more efficient to build systems using existing frameworks like Line Chain or other established options for enterprise needs.

LLMs (Large Language Models) are not extensively used in enterprise because in scenarios where a single retrieval system or model provider is already established and being utilized, the addition of a LLM like Lama Index may not provide significant value. It is often simpler and more efficient to build systems using existing frameworks like Line Chain or other established options for enterprise needs.


In [33]:
response = chat_engine.chat("What is Charlie's favorite restaurant?")
print(response)

Added user message to memory: What is Charlie's favorite restaurant?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"What is Charlie's favorite restaurant?"}
Got output: Patio Palo Alto.

Charlie's favorite restaurant is Patio Palo Alto.


In [34]:
response = chat_engine.chat("When is coffee chat? when is paper club? when are the office hours?")
print(response)

Added user message to memory: When is coffee chat? when is paper club? when are the office hours?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "When is coffee chat?"}
Got output: Coffee chat is held on Saturdays or Sundays, depending on the weekend.

=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "When is paper club?"}
Got output: Paper club is scheduled for Saturdays at 10 PM Pacific standard time.

=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "When are the office hours?"}
Got output: The office hours are scheduled to be 30 minutes before class, with the possibility of extending to an hour for certain topics not covered in the main syllabus.

- Coffee chat is held on Saturdays or Sundays, depending on the weekend.
- Paper club is scheduled for Saturdays at 10 PM Pacific standard time.
- The office hours are scheduled to be 30 minutes before class, with the possibility of extend

In [35]:
response = chat_engine.chat("What is the agenda for our next class?")
print(response)

Added user message to memory: What is the agenda for our next class?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"What is the agenda for our next class?"}
Got output: The agenda for the next class includes discussing frameworks, advanced civilization, advanced prompt engineering, introducing Ryke, focusing on Rag, talking about open source models with a specific mention of the Gemma model from Google, discussing agents and agentic behavior using the Crew AI framework, fine-tuning models, evaluating benchmarks, and exploring beyond text with multimodal inputs like image generation and video.

The agenda for our next class includes:
- Discussing frameworks
- Advanced civilization
- Advanced prompt engineering
- Introducing Ryke
- Focusing on Rag
- Talking about open source models with a specific mention of the Gemma model from Google
- Discussing agents and agentic behavior using the Crew AI framework
- Fine-tuning models
- Evaluating benchmarks
- Exp